In [1]:
import census_mapper
from zipfile import ZipFile
import io
import shapefile
from shapely.geometry import shape
import osr
import geopandas as gpd
import matplotlib.pyplot as plt
import csv
import numpy as np
import folium
import random
import pysal

In [98]:
results = {}
results2 = {}
with open("nov7results.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        results[row['Precinct']] = row['FP']
        results2[row['Precinct']] = row['Fort']

In [31]:
blocks = ZipFile('VTD2016-Shape.zip')
filenames = [y for y in sorted(blocks.namelist()) for ending in ['DBF', 'prj', 'shp', 'shx'] if y.endswith(ending)] 
dbf, prj, shp, shx = [io.BytesIO(blocks.read(filename)) for filename in filenames]

r = shapefile.Reader(shp=shp, shx=shx, dbf=dbf)

attributes, geometry = [], []
field_names = [field[0] for field in r.fields[1:]]  
for row in r.shapeRecords():  
    geometry.append(shape(row.shape.__geo_interface__))  
    attributes.append(dict(zip(field_names, row.record)))  

proj4_string = osr.SpatialReference(prj.read().decode('UTF-8')).ExportToProj4()
print(proj4_string)

blockgdf = gpd.GeoDataFrame(data = attributes, geometry = geometry, crs = proj4_string)
print(blockgdf.head())

+proj=longlat +ellps=GRS80 +no_defs 
       AREA COUNTY COUNTY_NAM   CTYNAME CTYNUMBER CTYSOSID  DATA DISTRICT  \
0  1.864348    036   COLUMBIA  Columbia        36   073131   690   073131   
1  1.636596    036   COLUMBIA  Columbia        36   073064   714   073064   
2  5.598643    036   COLUMBIA  Columbia        36   073061   711   073061   
3  0.872412    036   COLUMBIA  Columbia        36   073063   713   073063   
4  3.783726    036   COLUMBIA  Columbia        36   073132   691   073132   

   FIPS1 FIPS2       ID  POPULATION PRECINCT_I            PRECINCT_N  \
0  13073   073  6094217        1013        131  JOURNEY COMM. CHURCH   
1  13073   073  6094266        3858        064  GRACE BAPTIST CHURCH   
2  13073   073  6094297        1920        061       GREENBRIER HIGH   
3  13073   073  6094341        1770        063  RIVERSIDE ELEMENTARY   
4  13073   073  6094377        3131        132      WESLEY METHODIST   

                                            geometry  
0  POLYGON (

In [32]:
atl = ZipFile('Atlanta_City_Limits.zip')
filenames = [y for y in sorted(atl.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)] 
dbf, prj, shp, shx = [io.BytesIO(atl.read(filename)) for filename in filenames]

r = shapefile.Reader(shp=shp, shx=shx, dbf=dbf)

attributes, geometry = [], []
field_names = [field[0] for field in r.fields[1:]]  
for row in r.shapeRecords():  
    geometry.append(shape(row.shape.__geo_interface__))  
    attributes.append(dict(zip(field_names, row.record)))  

proj4_string = osr.SpatialReference(prj.read().decode('UTF-8')).ExportToProj4()
print(proj4_string)

atlgdf = gpd.GeoDataFrame(data = attributes, geometry = geometry, crs = proj4_string)
print(atlgdf.head())

+proj=longlat +ellps=WGS84 +no_defs 
     ACRES                                GLOBALID  ID    LABEL  OBJECTID  \
0  86643.1  {6999F328-8F07-4EE0-BF8C-4DB531A3898C}   0  Atlanta         1   

   OBJECTID_1    SHAPE_Area     SHAPE_Leng  SQ_MILES  \
0           1  5.087264e+08  195314.168371    135.38   

                                            geometry  
0  (POLYGON ((-84.42135081494511 33.6588738469827...  


In [110]:
blockgdf = blockgdf[blockgdf.geometry.is_valid == True]
fultongdf = blockgdf[blockgdf.geometry.intersects(atlgdf.geometry[0])]
fultongdf['FortP'] = list(0 for i in range(len(fultongdf.index)))
fultongdf['Fort'] = list(0 for i in range(len(fultongdf.index)))
fultongdf['FortA'] = list(0 for i in range(len(fultongdf.index)))
fultongdf['FortPC'] = list(0 for i in range(len(fultongdf.index)))
fultongdf['valid'] = list(1 for i in range(len(fultongdf.index)))

c:\program files (x86)\python35-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\program files (x86)\python35-32\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
c:\program files (x86)\python35-32\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [112]:
c = 0
for i, row in fultongdf.iterrows():
    pre = row['PRECINCT_N']
    if pre in results:
        if len(results[pre]) > 0:
            fultongdf.loc[i,'FortP'] = float(results[pre])
            fultongdf.loc[i,'Fort'] = float(results2[pre])
            fultongdf.loc[i,'FortA'] = float(results2[pre]) / row['AREA']
            if row['POPULATION'] == 0:
                fultongdf.loc[i, 'FortPC'] = 0
            else:
                fultongdf.loc[i,'FortPC'] = float(results2[pre]) / row['POPULATION']
            c += 1
    else:
        fultongdf.loc[i,'valid'] = 0

print(c)
fultongdf = fultongdf[fultongdf['valid'] == 1]

c:\program files (x86)\python35-32\lib\site-packages\pandas\core\indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


155


In [113]:
m = folium.Map(location=[33.75, -84.4], zoom_start=10.9, 
               tiles='Stamen Toner')

#threshold_scale = np.linspace(blockgdf['INCOME'].min(),
#                              blockgdf['INCOME'].max(), 6, dtype=int).tolist()
geo_str = (fultongdf).to_json()

#linear = folium.colormap.LinearColormap(['green', 'yellow', 'red'])
m.choropleth(geo_str=geo_str,
             data=fultongdf,columns=['ID','FortPC'],
             key_on='feature.properties.ID',
             fill_color="BuGn", 
             fill_opacity=0.8, line_opacity=0.8,
             legend_name='area')

#print(blockgo['Vincent Fort'])

atlgeo = atlgdf.to_json()
m.choropleth(geo_str=atlgeo, fill_color='None', line_color='black', line_weight=2)

m
m.save("FortPCVote.html")

In [ ]:
plt.clf()

fig, ax = plt.subplots(figsize=(10,10))

fultongdf
fultongdf.plot(ax = ax, column='FortP')


plt.xlim((-84.58,-84.28))
plt.ylim((33.62,33.92))

plt.show()

In [27]:
blocks = {}
with open("Income.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        blocks[row['Id2']] = [row['Median']]
print(len(blocks))
i = 0
with open("Race.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['Id2'] in blocks:
            blocks[row['Id2']] += [row['Estimate'], row['White'], row['Black'], row['Other']]
ageids = []
for i in range(50):
    text = 'HD01_VD'
    if i < 10:
        text += '0'    
    text += str(i)
    ageids += [text]
#print(ageids)
with open("Age.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        #print(row.keys())
        if row['GEO.id2'] in blocks:
            #print("hey")
            male = int(row['HD01_VD02'])
            female = int(row['HD01_VD26'])
            #print(ageids[7:13]+ageids[31:37])
            young = 0
            mid = 0
            old = 0
            for ind in ageids[7:13]+ageids[31:37]:
                young += int(row[ind])
            for ind in ageids[13:20]+ageids[37:44]:
                mid += int(row[ind])
            for ind in ageids[20:26]+ageids[44:]:
                old += int(row[ind])
            blocks[row['GEO.id2']] += [male, female, young, mid, old]
            #print(blocks[row['GEO.id2']])
for block in blocks:
    for i, ent in enumerate(blocks[block]):
        #print(ent)
        blocks[block][i] = int(str(ent).replace("+","").replace(",","").replace("-","0"))
    #for i, ent in enumerate(blocks[block]):
    #    if i > 1 and blocks[block][1] > 0:
    #        blocks[block][i] = 100 * blocks[block][i] / blocks[block][1]
        
#print(blocks)
#for geoid in blockgdf['geoid']

909


In [28]:
arr = []

for geoid in blockgdf.GEOID:
    #print(geoid)
    if str(geoid) in blocks:
        arr.append(blocks[str(geoid)])
    else:
        arr.append([0]*10)
    
#print(len(blockgdf.GEOID))
#print(len(arr))

arr = np.asarray(arr)

feats = ['INCOME', 'POP', 'WHITE', 'BLACK', 'OTHER_RACE', 'MALE', 'FEMALE', 'YOUNG', 'MIDAGE', 'OLD']

for i, feat in enumerate(feats):
    blockgdf[feat] = arr[:, i]
    blockgdf[feat+"A"] = arr[:, i]/(blockgdf.ALAND+blockgdf.AWATER)
    if i > 1:
        q = [0]*len(arr[:, i])
        for ind, a in enumerate(q):
            if arr[ind, 1] == 0:
                q[ind] = 0
            else:
                q[ind] = arr[ind, i]/arr[ind,1]
        blockgdf[feat+"P"] = q

candidates = {}
with open("poll.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        candidates[row['Candidate']] = [row['Total'], row['Young'], row['Mid'], row['Old'], row['White'], row['Black'], row['Other']]
        for i, c in enumerate(candidates[row['Candidate']]):
            candidates[row['Candidate']][i] = float(candidates[row['Candidate']][i])
#print(candidates)


blockgo = blockgdf[blockgdf.geometry.intersects(atlgdf.geometry[0])]
blockgo = blockgo[blockgo.POP > 0]
blockgo = blockgo[blockgo.GEOID != 131210103042]



tote = sum(blockgo['MALE']+blockgo['FEMALE'])

#participation = 

youngp = sum(blockgo['YOUNG'])
midagep = sum(blockgo['MIDAGE'])
oldp = sum(blockgo['OLD'])
whitep = sum(blockgo['WHITE'])
blackp = sum(blockgo['BLACK'])
otherp = sum(blockgo['OTHER_RACE'])
pops = [1, youngp, midagep, oldp, whitep, blackp, otherp]
print(pops)

candidates['Weight'] = [c/candidates['Weight'][0] for i, c in enumerate(candidates['Weight'])]

votes = 80000
turnout = [c*votes for c in candidates['Weight']]
turnoutrate = [t/pops[i] for i, t in enumerate(turnout)]
print(turnout)
print(turnoutrate)

print(candidates['Weight'])
for dude in candidates:
    for i in range(5):
        candidates[dude][i+1] *= turnoutrate[i+1]
    if not dude == "Weight":
        cage = [0]*len(blockgdf)
        crace = [0]*len(blockgdf)
        c = [0]*len(blockgdf)
        print(dude + str(candidates[dude]))
        for i in range(len(blockgdf)):
            if blockgdf['POP'][i] == 0:
                c[i] = 0
            else:
                #print(candidates[dude])
                cage[i] = candidates[dude][1]*blockgdf['YOUNG'][i]
                cage[i] += candidates[dude][2]*blockgdf['MIDAGE'][i]
                cage[i] += candidates[dude][3]*blockgdf['OLD'][i]
                #cage[i] = cage[i] / candidates['Weight'][0]
                crace[i] = candidates[dude][4]*blockgdf['WHITE'][i]
                crace[i] += candidates[dude][5]*blockgdf['BLACK'][i]
                crace[i] += candidates[dude][6]*blockgdf['OTHER_RACE'][i]
                #crace[i] = crace[i] / candidates['Weight'][0]
                #print(cage[i])
                #print(crace[i])
                cage[i] = cage[i] / blockgdf['POP'][i]
                crace[i] = crace[i] / blockgdf['POP'][i]
                c[i] = (cage[i] + crace[i])/200
                #print(c[i])
            
        #print(len(blockgo))
        #print(len(c))
        blockgdf[dude] = c
        
blockgo = blockgdf[blockgdf.geometry.intersects(atlgdf.geometry[0])]
blockgo = blockgo[blockgo.POP > 0]
blockgo = blockgo[blockgo.GEOID != 131210103042]



AttributeError: 'GeoDataFrame' object has no attribute 'GEOID'

In [29]:
m = folium.Map(location=[33.75, -84.4], zoom_start=11, 
               tiles='Stamen Toner')

#threshold_scale = np.linspace(blockgdf['INCOME'].min(),
#                              blockgdf['INCOME'].max(), 6, dtype=int).tolist()
geo_str = (blockgo).to_json()

#linear = folium.colormap.LinearColormap(['green', 'yellow', 'red'])
m.choropleth(geo_str=geo_str,
             data=blockgo,columns=['GEOID','Vincent Fort'],
             key_on='feature.properties.GEOID',
             fill_color="BuGn", 
             fill_opacity=0.8, line_opacity=0.8,
             legend_name='area')

#print(blockgo['Vincent Fort'])

atlgeo = atlgdf.to_json()
m.choropleth(geo_str=atlgeo, fill_color='None', line_color='black', line_weight=2)

m
#m.save("Map.html")

NameError: name 'blockgo' is not defined

In [30]:
plt.clf()

fig, ax = plt.subplots(figsize=(10,10))
blockgdf['rand'] = [random.randint(0, 10) for x in range(0, len(blockgdf))]
#print(blockgdf['rand'])

#blockgdf = blockgdf[blockgdf.GEOID != 131210103042]
#mask2 = 
#print(mask)

blockgo.plot(ax = ax, column='INCOME', edgecolor='grey', scheme='QUANTILES', legend=True)
#blockgdf[blockgdf.geometry.within(atlgdf.geometry[0])].plot(ax = ax)
atlgdf.plot(ax=ax, facecolor='none', alpha=0.4, ls='--')

plt.xlim((-84.58,-84.28))
plt.ylim((33.62,33.92))

plt.show()

NameError: name 'blockgo' is not defined